In [ ]:
import tensorflow as tf
import numpy as np
print(tf.__version__)

# 批量归一化

本节我们介绍批量归一化（batch normalization）层，它能让较深的神经网络的训练变得更加容易 [1]。在 “[实战Kaggle比赛：预测房价](http://zh.d2l.ai/chapter_deep-learning-basics/kaggle-house-price.html)” 一节里，我们对输入数据做了标准化处理：处理后的任意一个特征在数据集中所有样本上的均值为0、标准差为1。标准化处理输入数据使各个特征的分布相近：这往往更容易训练出有效的模型。

通常来说，数据标准化预处理对于浅层模型就足够有效了。随着模型训练的进行，当每层中参数更新时，靠近输出层的输出较难出现剧烈变化。但对深层神经网络来说，即使输入数据已做标准化，训练中模型参数的更新依然很容易造成靠近输出层输出的剧烈变化。这种计算数值的不稳定性通常令我们难以训练出有效的深度模型。

批量归一化的提出正是为了应对深度模型训练的挑战。在模型训练时，批量归一化利用小批量上的均值和标准差，不断调整神经网络中间输出，从而使整个神经网络在各层的中间输出的数值更稳定。批量归一化和下一节将要介绍的残差网络为训练和设计深度模型提供了两类重要思路。

## 5.10.1 批量归一化层

对全连接层和卷积层做批量归一化的方法稍有不同。下面我们将分别介绍这两种情况下的批量归一化。

### 5.10.1.1. 对全连接层做批量归一化
我们先考虑如何对全连接层做批量归一化。通常，我们将批量归一化层置于全连接层中的仿射变换和激活函数之间。设全连接层的输入为$\boldsymbol{u}$，权重参数和偏差参数分别为$\boldsymbol{W}$和$\boldsymbol{b}$，，激活函数为ϕ。设批量归一化的运算符为$\boldsymbol{BN}$。那么，使用批量归一化的全连接层的输出为：
$$\phi(\text{BN}(\boldsymbol{x})),$$
其中批量归一化输入由$\boldsymbol{x}$仿射变换
$$\boldsymbol{x} = \boldsymbol{W\boldsymbol{u} + \boldsymbol{b}}$$
得到。考虑一个由m个样本组成的小批量，仿射变换的输出为一个新的小批量$\mathcal{B} = \{\boldsymbol{x}^{(1)}, \ldots, \boldsymbol{x}^{(m)} \}$。它们正是批量归一化层的输入。对于小批量$\mathcal{B}$中任意样本$\boldsymbol{x}^{(i)} \in \mathbb{R}^d, 1 \leq i \leq m$，批量归一化层的输出同样是$\boldsymbol{d}$维向量
$$\boldsymbol{y}^{(i)} = \text{BN}(\boldsymbol{x}^{(i)}),$$
并由以下几步求得。首先，对小批量$\mathcal{B}$求均值和方差：
$$\boldsymbol{\mu}_\mathcal{B} \leftarrow \frac{1}{m}\sum_{i = 1}^{m} \boldsymbol{x}^{(i)},$$
$$\boldsymbol{\sigma}_\mathcal{B}^2 \leftarrow \frac{1}{m} \sum_{i=1}^{m}(\boldsymbol{x}^{(i)} - \boldsymbol{\mu}_\mathcal{B})^2,$$
其中的平方计算是按元素求平方。接下来，使用按元素开方和按元素除法对$\boldsymbol{x}^{(i)}$标准化：
$$\hat{\boldsymbol{x}}^{(i)} \leftarrow \frac{\boldsymbol{x}^{(i)} - \boldsymbol{\mu}_\mathcal{B}}{\sqrt{\boldsymbol{\sigma}_\mathcal{B}^2 + \epsilon}},$$
这里$\epsilon > 0$是一个很小的常数，保证分母大于0。在上面标准化的基础上，批量归一化层引入了两个可以学习的模型参数，拉伸（scale）参数$\boldsymbol{\gamma}$和偏移（shift）参数$\boldsymbol{\beta}$。这两个参数和$\boldsymbol{x}^{(i)}$形状相同，皆为d维向量。它们与$\hat{\boldsymbol{x}}^{(i)}$分别做按元素乘法（符号$\odot$）和加法计算：
$${\boldsymbol{y}}^{(i)} \leftarrow \boldsymbol{\gamma} \odot \hat{\boldsymbol{x}}^{(i)} + \boldsymbol{\beta}.$$
至此，我们得到了$\boldsymbol{x}^{(i)}$的批量归一化的输出$\boldsymbol{y}^{(i)}$。 值得注意的是，可学习的拉伸和偏移参数保留了不对$\boldsymbol{x}^{(i)}$做批量归一化的可能：此时只需学出$\boldsymbol{\gamma} = \sqrt{\boldsymbol{\sigma}_\mathcal{B}^2 + \epsilon}$和$\boldsymbol{\beta} = \boldsymbol{\mu}_\mathcal{B}$。我们可以对此这样理解：如果批量归一化无益，理论上，学出的模型可以不使用批量归一化。

### 5.10.1.2. 对卷积层做批量归一化
对卷积层来说，批量归一化发生在卷积计算之后、应用激活函数之前。如果卷积计算输出多个通道，我们需要对这些通道的输出分别做批量归一化，且每个通道都拥有独立的拉伸和偏移参数，并均为标量。设小批量中有 m个样本。在单个通道上，假设卷积计算输出的高和宽分别为 p和 q 。我们需要对该通道中 m×p×q个元素同时做批量归一化。对这些元素做标准化计算时，我们使用相同的均值和方差，即该通道中 m×p×q个元素的均值和方差。

### 5.10.1.3. 预测时的批量归一化
使用批量归一化训练时，我们可以将批量大小设得大一点，从而使批量内样本的均值和方差的计算都较为准确。将训练好的模型用于预测时，我们希望模型对于任意输入都有确定的输出。因此，单个样本的输出不应取决于批量归一化所需要的随机小批量中的均值和方差。一种常用的方法是通过移动平均估算整个训练数据集的样本均值和方差，并在预测时使用它们得到确定的输出。可见，和丢弃层一样，批量归一化层在训练模式和预测模式下的计算结果也是不一样的。

## 5.10.2. 从零开始实现
下面我们通过numpy中的ndarray来实现批量归一化层。

这段代码实现了批量归一化(Batch Normalization)操作，是深度学习中一个重要的网络层。让我逐步解释它的功能：

1. 函数接收以下参数：
   - is_training：标识是训练模式还是预测模式
   - X：输入数据
   - gamma：缩放参数
   - beta：偏移参数
   - moving_mean：移动平均的均值
   - moving_var：移动平均的方差
   - eps：一个很小的数，防止除零
   - momentum：动量参数，用于移动平均的更新

2. 函数的主要逻辑分两种模式：

   预测模式 (is_training = False):
   - 直接使用已经计算好的移动平均均值和方差来归一化数据

   训练模式 (is_training = True):
   - 计算当前批次的均值和方差
   - 支持两种数据形状：
     * 二维数据(全连接层的情况)：在特征维度上计算统计量
     * 四维数据(卷积层的情况)：在通道维度上计算统计量
   - 使用当前批次的统计量进行归一化
   - 更新移动平均的均值和方差

3. 最后的步骤：
   - 对归一化后的数据进行缩放和偏移（使用gamma和beta参数）
   - 返回处理后的数据和更新后的移动平均值

批量归一化的主要目的是：
- 减少内部协变量偏移
- 加速网络训练
- 允许使用更大的学习率
- 减少对初始化的依赖
- 具有轻微正则化效果


In [2]:
def batch_norm(is_training,X, gamma, beta, moving_mean, moving_var, eps, momentum):
    # 判断是当前模式是训练模式还是预测模式
    if not is_training:
        # 如果是在预测模式下，直接使用传入的移动平均所得的均值和方差
        X_hat = (X - moving_mean) / np.sqrt(moving_var + eps)
    else:
        assert len(X.shape) in (2, 4)
        if len(X.shape) == 2:
            # 使用全连接层的情况，计算特征维上的均值和方差
            mean = X.mean(axis=0)
            var = ((X - mean) ** 2).mean(axis=0)
        else:
            # 使用二维卷积层的情况，计算通道维上（axis=1）的均值和方差。这里我们需要保持
            # X的形状以便后面可以做广播运算
            mean = X.mean(axis=(0, 2, 3), keepdims=True)
            var = ((X - mean) ** 2).mean(axis=(0, 2, 3), keepdims=True)
        # 训练模式下用当前的均值和方差做标准化
        X_hat = (X - mean) / np.sqrt(var + eps)
        # 更新移动平均的均值和方差
        moving_mean = momentum * moving_mean + (1.0 - momentum) * mean
        moving_var = momentum * moving_var + (1.0 - momentum) * var
    Y = gamma * X_hat + beta  # 拉伸和偏移
    return Y, moving_mean, moving_var

接下来，我们自定义一个BatchNorm层。它保存参与求梯度和迭代的拉伸参数gamma和偏移参数beta，同时也维护移动平均得到的均值和方差，以便能够在模型预测时被使用。BatchNorm实例所需指定的num_features参数对于全连接层来说应为输出个数，对于卷积层来说则为输出通道数。该实例所需指定的num_dims参数对于全连接层和卷积层来说分别为2和4。

自定义的 BatchNormalization 层

1. 类的初始化（__init__）:
   - 接收两个主要参数：decay（衰减率，默认0.9）和 epsilon（防止除零的小数值，默认1e-5）
   - 继承自 tf.keras.layers.Layer 基类

2. 构建层（build 方法）:
   创建四个关键参数：
   - gamma：可训练的缩放参数，初始化为1
   - beta：可训练的偏移参数，初始化为0
   - moving_mean：不可训练的移动平均均值，初始化为0
   - moving_variance：不可训练的移动平均方差，初始化为1

3. 移动平均计算（assign_moving_average 方法）:
   - 实现指数移动平均的更新
   - 公式：new_value = old_value * decay + current_value * (1 - decay)

4. 核心计算逻辑（call 方法）:
   分为训练和推理两种模式：
   
   训练模式：
   - 计算当前批次的均值和方差
   - 更新移动平均值
   - 使用当前批次的统计量进行归一化

   推理模式：
   - 直接使用已经计算好的移动平均值
   
   最后：
   - 使用 tf.nn.batch_normalization 进行标准化操作
   - 应用 gamma（缩放）和 beta（偏移）参数

5. 输出形状计算（compute_output_shape 方法）:
   - 保持输入形状不变

这个实现展示了批量归一化的核心原理：
- 在训练时对每个批次进行归一化
- 维护移动平均统计量用于推理
- 包含可学习的缩放和偏移参数
- 区分训练和推理两种模式的不同行为

gamma、beta、moving_mean和moving_variance这四个参数在batch normalization中的作用

1. **gamma (缩放参数)**
   - 是一个可训练参数
   - 用途：控制归一化后数据的缩放程度
   - 作用：
     * 让网络可以学习恢复数据的原始分布（如果需要的话）
     * 如果gamma=1，保持标准化的尺度
     * 如果gamma>1，扩大数据分布范围
     * 如果gamma<1，压缩数据分布范围
   - 初始化通常设为1

2. **beta (偏移参数)**
   - 是一个可训练参数
   - 用途：控制归一化后数据的偏移量
   - 作用：
     * 让网络可以学习数据的最优偏移量
     * 可以整体平移数据分布
     * 补偿归一化可能带来的数据分布改变
   - 初始化通常设为0

3. **moving_mean (移动平均均值)**
   - 是一个不可训练参数
   - 用途：记录训练过程中所有批次的均值的指数移动平均
   - 作用：
     * 在推理阶段使用，提供稳定的归一化参数
     * 消除批次间的波动影响
     * 使预测结果更加稳定
   - 初始化为0，在训练过程中不断更新

4. **moving_variance (移动平均方差)**
   - 是一个不可训练参数
   - 用途：记录训练过程中所有批次的方差的指数移动平均
   - 作用：
     * 在推理阶段使用，提供稳定的归一化参数
     * 消除批次间的波动影响
     * 使预测结果更加稳定
   - 初始化为1，在训练过程中不断更新

这些参数的协同作用：
1. **训练阶段**：
   - 使用当前批次的均值和方差进行归一化
   - 同时更新moving_mean和moving_variance
   - 应用gamma和beta进行缩放和偏移

2. **推理阶段**：
   - 使用moving_mean和moving_variance进行归一化
   - 应用训练好的gamma和beta进行缩放和偏移

这种设计的优点：
- 保持了网络的表达能力
- 提供了稳定的推理结果
- 允许模型学习最优的数据分布
- 解决了内部协变量偏移问题

总的来说，这四个参数共同确保了batch normalization既能在训练时提供良好的正则化效果，又能在推理时保持稳定的输出。

### 让我用简单的方式解释"偏移量"：

想象一下在坐标系中的一条曲线：
1. 当你给这条曲线加上一个正的beta值，整条曲线会向上移动
2. 当你给这条曲线加上一个负的beta值，整条曲线会向下移动

举个具体的例子：
- 假设经过标准化后，某个神经元的输出值是1.0
- 如果beta = 0.5，那么最终输出就变成了1.5
- 如果beta = -0.5，那么最终输出就变成了0.5

简单来说，beta就像是一个"调整器"：
- 它可以整体提升或降低数据的数值
- 这种调整是统一的，对所有数据都加上相同的值
- 这样的调整有助于网络找到数据的最佳位置

用生活中的例子：
- 如果把数据比作电视的画面
- beta就像是画面的亮度调节
- 可以让整个画面统一变亮或变暗

所以，"偏移量"就是整体平移数据的一个值，让数据整体上移或下移。

### 用非技术的语言介绍batch normalization

让我用生活化的例子来解释Batch Normalization：

想象你是一个老师，负责评分：

1. **标准化过程**
- 一个班级100人参加考试，分数分布很不均匀，有的98分，有的30分
- 为了更好地评估学生水平，你决定"标准化"这些分数：
  * 先算出班级平均分（比如70分）
  * 看看每个人比平均分高或低多少
  * 最后把分数调整到一个合理的范围内

2. **为什么要这样做？**
- 让评分更公平：不同难度的考试可以比较
- 更容易发现问题：清楚地看出谁在平均水平之上或之下
- 便于后续处理：调整后的分数更容易进行进一步分析

3. **moving_mean和moving_variance的作用**
就像老师会记录：
- 这个班级历次考试的平均水平
- 不会只看一次考试的情况
- 通过多次考试的记录，更准确地了解班级整体水平

4. **gamma和beta的作用**
相当于老师可以：
- 根据需要调整分数的分布范围（gamma）
- 适当提高或降低整体分数（beta）
- 使最终的分数更合理，更有意义

5. **训练和预测的区别**
- 训练时：就像平时考试，每次都要算当次的平均分和分布
- 预测时：就像期末总评，要参考学生在整个学期的表现

这种方法的好处：
- 让评分更稳定
- 更容易发现学生的真实水平
- 便于跟其他班级比较
- 有助于后续的教学调整

这就是Batch Normalization的基本思想：通过标准化和适当的调整，让数据更容易被处理，最终得到更好的结果。

### 用非技术语言描述gamma参数如何使用和训练

让我用一个体育课测试的例子来解释gamma参数：

想象你是体育老师，在测量学生的跳远成绩：

1. **基本情况**
- 已经把所有学生的成绩标准化了（比如都调整到0-1之间）
- gamma就像是一个"放大镜"或"缩小镜"

2. **gamma的作用**
- gamma = 1: 保持现状，成绩就维持标准化后的样子
- gamma > 1: 放大差异，比如：
  * 原本相差0.1分的两个成绩
  * 可能会被放大到相差0.3分
- gamma < 1: 缩小差异，比如：
  * 原本相差0.1分的两个成绩
  * 可能会被缩小到相差0.03分

3. **gamma如何训练**
就像老师在不断调整评分标准：
- 一开始设定gamma = 1
- 通过观察学生的表现，逐渐调整：
  * 如果发现成绩差异太小，不容易区分，就增大gamma
  * 如果发现成绩差异太大，不够合理，就减小gamma
- 这个调整是自动的，系统会根据需要找到最合适的值

4. **为什么需要gamma**
就像在评分时：
- 有时需要放大差异，让优秀和不足更明显
- 有时需要缩小差异，让评分更温和
- gamma提供了这种灵活性

简单说：
- gamma就是一个可以自动调节的"放大镜"
- 它帮助系统找到最合适的区分度
- 这个过程是自动的，不需要人工干预

### 使用非技术语言描述beta参数如何作用以及训练

让我继续用体育课测试的例子来解释beta参数：

1. **基本情况**
想象你是体育老师，在给跳远成绩评分：
- 所有成绩已经标准化了（假设都在0分附近）
- beta就像是一个"整体加分"或"整体减分"的工具

2. **beta的作用**
就像调整基础分：
- beta = 0：保持原样，不加不减
- beta = 2：所有人都加2分
- beta = -1：所有人都减1分

举个具体例子：
- 小明原始分数是0.5
- 如果beta = 2，小明最终得分就是2.5
- 如果beta = -1，小明最终得分就是-0.5
- 这种加减应用于所有学生

3. **beta如何训练**
就像老师在不断调整基础分：
- 一开始设定beta = 0（不加不减）
- 系统会自动观察并调整：
  * 如果发现分数整体偏低，就增加beta
  * 如果发现分数整体偏高，就减少beta
- 这个过程是自动的，系统会找到最合适的加减分值

4. **为什么需要beta**
就像在实际评分中：
- 有时题目太难，需要适当加分
- 有时题目太简单，需要适当减分
- beta提供了整体调节的能力

简单说：
- beta就像一个"统一加减分"的工具
- 它能整体提升或降低所有分数
- 这个加减分的数值是自动学习得到的
- 目的是让最终分数更合理

生活中的类比：
- 就像调整电视机的亮度
- beta就是让画面整体变亮或变暗的旋钮
- 系统会自动找到最舒服的亮度水平


总结一下我们讨论的要点：
1. Batch Normalization 就像是一个智能的评分系统
2. gamma 像是一个自动调节的"放大镜"，控制差异的程度
3. beta 像是一个"整体加减分"的工具，进行统一调整


In [3]:
# 自定义BatchNormalization层，代码运行不起来，暂时跳过
# 仅用来理解原理
class BatchNormalization(tf.keras.layers.Layer):
    def __init__(self, decay=0.9, epsilon=1e-5, **kwargs):
        self.decay = decay
        self.epsilon = epsilon
        super(BatchNormalization, self).__init__(**kwargs)

    def build(self, input_shape):
        self.gamma = self.add_weight(name='gamma',
                                     shape=[input_shape[-1], ],
                                     initializer=tf.initializers.ones,
                                     trainable=True)
        self.beta = self.add_weight(name='beta',
                                    shape=[input_shape[-1], ],
                                    initializer=tf.initializers.zeros,
                                    trainable=True)
        self.moving_mean = self.add_weight(name='moving_mean',
                                           shape=[input_shape[-1], ],
                                           initializer=tf.initializers.zeros,
                                           trainable=False)
        self.moving_variance = self.add_weight(name='moving_variance',
                                               shape=[input_shape[-1], ],
                                               initializer=tf.initializers.ones,
                                               trainable=False)
        super(BatchNormalization, self).build(input_shape)

    def assign_moving_average(self, variable, value):
        """
        variable = variable * decay + value * (1 - decay)
        """
        delta = variable * self.decay + value * (1 - self.decay)
        return variable.assign(delta)

    @tf.function
    def call(self, inputs, training):
        if training:
            batch_mean, batch_variance = tf.nn.moments(inputs, list(range(len(inputs.shape) - 1)))
            mean_update = self.assign_moving_average(self.moving_mean, batch_mean)
            variance_update = self.assign_moving_average(self.moving_variance, batch_variance)
            self.add_update(mean_update)
            self.add_update(variance_update)
            mean, variance = batch_mean, batch_variance
        else:
            mean, variance = self.moving_mean, self.moving_variance
        output = tf.nn.batch_normalization(inputs,
                                           mean=mean,
                                           variance=variance,
                                           offset=self.beta,
                                           scale=self.gamma,
                                           variance_epsilon=self.epsilon)
        return output

    def compute_output_shape(self, input_shape):
        return input_shape

### 5.10.2.1. 使用批量归一化层的LeNet

下面我们修改“[卷积神经网络（LeNet）](http://zh.d2l.ai/chapter_convolutional-neural-networks/lenet.html)”这一节介绍的LeNet模型，从而应用批量归一化层。我们在所有的卷积层或全连接层之后、激活层之前加入批量归一化层。

### 添加Batch Normalization后的LeNet网络模型

1. **输入层**
- 接收28×28大小的灰度图片（1个通道）

2. **第一个卷积块**
- 使用6个5×5的卷积核扫描图片
- 用Batch Normalization进行数据标准化
- 使用sigmoid激活函数增加非线性
- 用2×2的最大池化层缩小特征图尺寸

3. **第二个卷积块**
- 使用16个5×5的卷积核进行特征提取
- 同样进行Batch Normalization标准化
- 使用sigmoid激活函数
- 再次用2×2的最大池化层减小尺寸

4. **扁平化操作**
- 将二维特征图展平成一维向量

5. **第一个全连接层**
- 连接到120个神经元
- 使用Batch Normalization
- 使用sigmoid激活函数

6. **第二个全连接层**
- 连接到84个神经元
- 使用Batch Normalization
- 使用sigmoid激活函数

7. **输出层**
- 最后输出10个数值（可能用于10分类问题）
- 使用sigmoid激活函数

主要特点：
- 每个主要层后都加入了Batch Normalization，有助于训练
- 使用了典型的"卷积+池化"结构提取特征
- 全部使用sigmoid激活函数
- 整体结构是"窄而深"的，逐层提取更抽象的特征

这个网络结构适合处理简单的图像分类任务，比如MNIST手写数字识别。

### 计算该神经网络模型每一层的训练参数

1. **第一个卷积层**
- 卷积核：5×5×1×6
- 参数量 = (5×5×1)×6 + 6(偏置) = 156个参数

2. **第一个Batch Normalization层**
- 每个通道有gamma和beta两个参数
- 参数量 = 6×2 = 12个参数

3. **第二个卷积层**
- 卷积核：5×5×6×16
- 参数量 = (5×5×6)×16 + 16(偏置) = 2,416个参数

4. **第二个Batch Normalization层**
- 每个通道有gamma和beta两个参数
- 参数量 = 16×2 = 32个参数

5. **第一个全连接层（Dense-120）**
- 输入是扁平化后的特征图
- 假设输入大小为N（需要计算）
- 参数量 = N×120 + 120(偏置)
- [需要计算输入尺寸N]

6. **第三个Batch Normalization层**
- 参数量 = 120×2 = 240个参数

7. **第二个全连接层（Dense-84）**
- 参数量 = 120×84 + 84(偏置) = 10,164个参数

8. **第四个Batch Normalization层**
- 参数量 = 84×2 = 168个参数

9. **输出层（Dense-10）**
- 参数量 = 84×10 + 10(偏置) = 850个参数

为了计算完整的参数量，我们需要计算第一个全连接层的输入尺寸：
1. 输入图片：28×28
2. 第一次卷积(5×5)：24×24×6
3. 第一次池化(2×2)：12×12×6
4. 第二次卷积(5×5)：8×8×16
5. 第二次池化(2×2)：4×4×16
6. 扁平化后：4×4×16 = 256

所以第一个全连接层的参数量：
- 参数量 = 256×120 + 120 = 30,840个参数

总参数量：
156 + 12 + 2,416 + 32 + 30,840 + 240 + 10,164 + 168 + 850 = 44,878个参数

请注意：
- BN层还有两个不可训练参数（moving_mean和moving_variance）
- 实际使用时可以用model.summary()查看准确的参数量

In [4]:
inputs = tf.keras.Input(shape=(28, 28, 1))
x = tf.keras.layers.Conv2D(filters=6, kernel_size=5)(inputs)
x = BatchNormalization()(x, training=True)
x = tf.keras.layers.Activation('sigmoid')(x)
x = tf.keras.layers.MaxPool2D(pool_size=2, strides=2)(x)
x = tf.keras.layers.Conv2D(filters=16, kernel_size=5)(x)
x = BatchNormalization()(x, training=True)
x = tf.keras.layers.Activation('sigmoid')(x)
x = tf.keras.layers.MaxPool2D(pool_size=2, strides=2)(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(120)(x)
x = BatchNormalization()(x, training=True)
x = tf.keras.layers.Activation('sigmoid')(x)
x = tf.keras.layers.Dense(84)(x)
x = BatchNormalization()(x, training=True)
x = tf.keras.layers.Activation('sigmoid')(x)
outputs = tf.keras.layers.Dense(10, activation='sigmoid')(x)

net = tf.keras.Model(inputs=inputs, outputs=outputs)

下面我们训练修改后的模型。

In [5]:
# (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
# x_train = x_train.reshape((60000, 28, 28, 1)).astype('float32') / 255
# x_test = x_test.reshape((10000, 28, 28, 1)).astype('float32') / 255

# net.compile(loss='sparse_categorical_crossentropy',
#               optimizer=tf.keras.optimizers.RMSprop(),
#               metrics=['accuracy'])
# history = net.fit(x_train, y_train,
#                     batch_size=64,
#                     epochs=5,
#                     validation_split=0.2)
# test_scores = net.evaluate(x_test, y_test, verbose=2)
# print('Test loss:', test_scores[0])
# print('Test accuracy:', test_scores[1])

最后我们查看第一个批量归一化层学习到的拉伸参数gamma和偏移参数beta。

In [6]:
# net.get_layer(index=1).gamma,net.get_layer(index=1).beta

## 5.10.3 简洁实现

与我们刚刚自己定义的`BatchNorm`类相比，keras中`layers`模块定义的`BatchNorm`类使用起来更加简单。它不需要指定自己定义的`BatchNorm`类中所需的`num_features`和`num_dims`参数值。在keras中，这些参数值都将通过延后初始化而自动获取。下面我们用keras实现使用批量归一化的LeNet。

In [7]:
net = tf.keras.models.Sequential()
net.add(tf.keras.layers.Conv2D(filters=6,kernel_size=5))
net.add(tf.keras.layers.BatchNormalization())
net.add(tf.keras.layers.Activation('sigmoid'))
net.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
net.add(tf.keras.layers.Conv2D(filters=16,kernel_size=5))
net.add(tf.keras.layers.BatchNormalization())
net.add(tf.keras.layers.Activation('sigmoid'))
net.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
net.add(tf.keras.layers.Flatten())
net.add(tf.keras.layers.Dense(120))
net.add(tf.keras.layers.BatchNormalization())
net.add(tf.keras.layers.Activation('sigmoid'))
net.add(tf.keras.layers.Dense(84))
net.add(tf.keras.layers.BatchNormalization())
net.add(tf.keras.layers.Activation('sigmoid'))
net.add(tf.keras.layers.Dense(10,activation='sigmoid'))

使用同样的超参数进行训练。

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape((60000, 28, 28, 1)).astype('float32') / 255
x_test = x_test.reshape((10000, 28, 28, 1)).astype('float32') / 255

net.compile(loss='sparse_categorical_crossentropy',
            optimizer=tf.keras.optimizers.RMSprop(),
            metrics=['accuracy'])
history = net.fit(x_train, y_train,
                  batch_size=64,
                  epochs=5,
                  validation_split=0.2)
test_scores = net.evaluate(x_test, y_test, verbose=2)
print('Test loss:', test_scores[0])
print('Test accuracy:', test_scores[1])

## 5.10.4 小结

* 在模型训练时，批量归一化利用小批量上的均值和标准差，不断调整神经网络的中间输出，从而使整个神经网络在各层的中间输出的数值更稳定。
* 对全连接层和卷积层做批量归一化的方法稍有不同。
* 批量归一化层和丢弃层一样，在训练模式和预测模式的计算结果是不一样的。
* keras提供的BatchNorm类使用起来简单、方便。


## 5.10.4 练习

* 能否将批量归一化前的全连接层或卷积层中的偏差参数去掉？为什么？（提示：回忆批量归一化中标准化的定义。）
* 尝试调大学习率。同[“卷积神经网络（LeNet）”](lenet.ipynb)一节中未使用批量归一化的LeNet相比，现在是不是可以使用更大的学习率？
* 尝试将批量归一化层插入LeNet的其他地方，观察并分析结果的变化。
* 尝试一下不学习拉伸参数`gamma`和偏移参数`beta`（构造的时候加入参数`grad_req='null'`来避免计算梯度），观察并分析结果。
* 查看`BatchNorm`类的文档来了解更多使用方法，例如，如何在训练时使用基于全局平均的均值和方差。




## 5.10.4 参考文献

[1] Ioffe, S., & Szegedy, C. (2015). Batch normalization: Accelerating deep network training by reducing internal covariate shift. arXiv preprint arXiv:1502.03167.